In [1]:
import pandas as pd
import json
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from Data_Preprocess_Cleaning import spacy_cleaner

threshold1 = 0.1  # 0.156
threshold2 = 0.282

model1_path = './models/evidence_identification_model/model.epoch00-loss1.22'
model2_path = './models/evidence_classification_model/model.epoch02-loss0.93'
model_checkpoint_distilbert = "distilbert-base-uncased"
model_checkpoint_bert = "bert-base-uncased"

C:\ProgramData\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model1 = TFAutoModelForSequenceClassification.from_pretrained(model1_path,num_labels = 2)
tokenizer1 = AutoTokenizer.from_pretrained(model_checkpoint_distilbert)
model2 = TFAutoModelForSequenceClassification.from_pretrained(model2_path,num_labels = 2)
tokenizer2 = AutoTokenizer.from_pretrained(model_checkpoint_bert)

def predict_proba(rumor, timeline, max_length, debug = 0):
    tokenizer_output = tokenizer1(rumor, timeline, truncation=True, max_length = max_length, return_tensors = 'tf', padding=True)
    logits = model1(**tokenizer_output)["logits"]
    predicted_proba_class_id = tf.nn.softmax(logits, axis=1).numpy()[0][1]
    return predicted_proba_class_id

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at ./models/evidence_identification_model/model.epoch00-loss1.22.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
C:\ProgramData\miniconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some layers from the model checkpoint at ./models/evidence_classification_model/model.epoch02-loss0.93 were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from 

In [3]:
# def inference_fn(data):
#     rumor_id = data['id']
#     rumor = data['rumor']

#     username = [entry[0] for entry in data['timeline']]
#     timeline_id = [entry[1] for entry in data['timeline']]
#     timelines = [entry[2] for entry in data['timeline']]

#     predictions = [predict_proba(spacy_cleaner(rumor), spacy_cleaner(timeline), max_length=192) for timeline in timelines]

#     tdf = pd.DataFrame()
#     tdf['username'] = username
#     tdf['timeline_id'] = timeline_id
#     tdf['timelines'] = timelines
#     tdf['predictions'] = predictions
#     tdf['id'] = rumor_id
#     tdf['rumor'] = rumor

#     tdf = tdf.sort_values(by=['predictions'], ascending=False).reset_index(drop=True)
#     tdf = tdf[tdf['predictions']>threshold1]

#     if tdf.shape[0]==0:
#         predictions_final = "NOT ENOUGH INFO"
#         predicted_evidence = ""
#     else:
#         if (tdf.shape[0]>5):
#             tdf = tdf.iloc[0:5]

#         tdf['rank'] = tdf.index + 1
#         tdf['Dummy'] = 'A0'
#         tdf['Team'] = 'KNK'

#         EVLIST.extend(tdf[['id', 'Dummy', 'timeline_id', 'rank', 'predictions', 'Team']].values.tolist())

#         predicted_evidence = tdf[['username', 'timeline_id', 'timelines']].values.tolist()

#         evidence = " ".join(tdf['timelines'].values.tolist())


#         predictions_final = "SUPPORTS" if predict_proba(spacy_cleaner(rumor), spacy_cleaner(evidence), max_length=512) >= threshold2 else "REFUTES"
        
#         return tdf[['id', 'Dummy', 'timeline_id', 'rank', 'predictions', 'Team']].values.tolist(), [rumor_id, predictions_final, rumor, predicted_evidence]

#     return [[]], [rumor_id, predictions_final, rumor, predicted_evidence]


# EVLIST = []
# FLIST = []

# with open('./data/original_data/English_test.json', 'rb') as f:
#     for line in f:
#         data = json.loads(line)

#         print(data.keys())
        
#         out1, out2 = inference_fn(data)
#         if len(out1)>0:
#             EVLIST.extend(out1)
#         if len(FLIST)>0:
#             FLIST.append(out2)
# Evidence_retrieval_output = pd.DataFrame(EVLIST, columns = ['id', 'Dummy', 'timeline_id', 'rank', 'predictions', 'Team'])
# Evidence_retrieval_output.to_csv('./results/Evidence_retrieval_output.csv', index=False)
# print(Evidence_retrieval_output)
# Rumor_Verification_output = pd.DataFrame(FLIST, columns = ['id', 'predicted_label', 'claim', 'predicted_evidence'])
# Rumor_Verification_output.to_csv('./results/Rumor_Verification_output.csv', index=False)
# print(Rumor_Verification_output)

In [4]:
EVLIST = []
FLIST = []
with open('./data/original_data/English_test.json', 'rb') as f:
    for line in f:
        data = json.loads(line)

        print(data.keys())
        
        rumor_id = data['id']
        rumor = data['rumor']
        
        username = [entry[0] for entry in data['timeline']]
        timeline_id = [entry[1] for entry in data['timeline']]
        timelines = [entry[2] for entry in data['timeline']]
        
        predictions = [predict_proba(spacy_cleaner(rumor), spacy_cleaner(timeline), max_length=192) for timeline in timelines]
        
        tdf = pd.DataFrame()
        tdf['username'] = username
        tdf['timeline_id'] = timeline_id
        tdf['timelines'] = timelines
        tdf['predictions'] = predictions
        tdf['id'] = rumor_id
        tdf['rumor'] = rumor
        
        tdf = tdf.sort_values(by=['predictions'], ascending=False).reset_index(drop=True)
        
        if tdf.shape[0]==0:
            predicted_evidence = ""
        else:
            if tdf.shape[0]>5:
                tdff = tdf.iloc[0:5]
            else:
                tdff = tdf
            predicted_evidence = tdff[['username', 'timeline_id', 'timelines']].values.tolist()
        
        tdf = tdf[tdf['predictions']>threshold1]

        if tdf.shape[0]==0:
            predictions_final = "NOT ENOUGH INFO"
            
        else:
            if (tdf.shape[0]>5):
                tdf = tdf.iloc[0:5]
        
            tdf['rank'] = tdf.index + 1
            tdf['Dummy'] = 'A0'
            tdf['Team'] = 'KNK'

            EVLIST.extend(tdf[['id', 'Dummy', 'timeline_id', 'rank', 'predictions', 'Team']].values.tolist())

            predicted_evidence = tdf[['username', 'timeline_id', 'timelines']].values.tolist()

            evidence = " ".join(tdf['timelines'].values.tolist())


            predictions_final = "SUPPORTS" if predict_proba(spacy_cleaner(rumor), spacy_cleaner(evidence), max_length=512) >= threshold2 else "REFUTES"

        FLIST.append([rumor_id, predictions_final, rumor, predicted_evidence])
        
        print(EVLIST)
        print(FLIST)

dict_keys(['id', 'rumor', 'timeline'])
[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK']]
[['AuRED_139', 'SUPPORTS', 'The leak of the mouse in the Al-Ahly and El-Gouna match, the referees’ admission of fraud. https://t.co/aB1EXKjzuk', [['https://twitter.com/EFA', '1551966453804531712', '🔴 Media Statement 🇪🇬 The Egyptian Football Association confirms that the audio leak that recently spread regarding the Al-Ahly and El-Gouna match in the twenty-sixth week of the Egyptian League is fake and has no basis in truth. https://t.co/PX2VOAR2HA']]]]
dict_keys(['id', 'rumor', 'timeline'])
[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK']]
[['AuRED_139', 'SUPPORTS', 'The leak of the mouse in the Al-Ahly and El-Gouna match, the referees’ admission of fraud. https://t.co/aB1EXKjzuk', [['https://twitter.com/EFA', '1551966453804531712', '🔴 Media Statement 🇪🇬 The Egyptian Football Association confirms that the audio leak that recently spread regarding the

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK']]
[['AuRED_139', 'SUPPORTS', 'The leak of the mouse in the Al-Ahly and El-Gouna match, the referees’ admission of fraud. https://t.co/aB1EXKjzuk', [['https://twitter.com/EFA', '1551966453804531712', '🔴 Media Statement 🇪🇬 The Egyptian Football Association confirms that the audio leak that recently spread regarding the Al-Ahly and El-Gouna match in the twenty-sixth week of the Egyptian League is fake and has no basis in truth. https://t.co/PX2VOAR2HA']]], ['AuRED_151', 'NOT ENOUGH INFO', 'Journalist Samer Jawad: Millions of dollars disappeared from t

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK']]
[['AuRED_139', 'SUPPORTS', 'The leak of the mouse in the Al-Ahly and El-Gouna match, the referees’ admission of fraud. https://t.co/aB1EXKjzuk', [['https://twitter.com/EFA', '1551966453804531712', '🔴 Media Statement 🇪🇬 The Egyptian Football Association confirms that the audio leak that recently spread regarding the Al-Ahly and El-Gouna match in the twenty-sixth week of the Egyptian League is fake and has no basis in truth. https://t.co/PX2VOAR2HA']]], ['AuRED_151', 'NOT ENO

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK'], ['AuRED_040', 'A0', '1266672477272408064', 1, 0.1513330191373825, 'KNK'], ['AuRED_040', 'A0', '1265649630110507010', 2, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1265626365400961024', 3, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1266324977382555650', 4, 0.12719853222370148, 'KNK'], ['AuRED_040', 'A0', '1266483508299608072', 5, 0.1193590760231018, 'KNK']]
[['AuRED_139', 'SUPPORTS', 'The leak of the mouse in the Al-Ahly and El-Gouna match, the referees’ admiss

dict_keys(['id', 'rumor', 'timeline'])
[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK'], ['AuRED_040', 'A0', '1266672477272408064', 1, 0.1513330191373825, 'KNK'], ['AuRED_040', 'A0', '1265649630110507010', 2, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1265626365400961024', 3, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1266324977382555650', 4, 0.12719853222370148, 'KNK'], ['AuRED_040', 'A0', '1266483508299608072', 5, 0.1193590760231018, 'KNK'], ['AuRED_067', 'A0', '1292482758560948224', 1, 0.18775562942028046,

...

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK'], ['AuRED_040', 'A0', '1266672477272408064', 1, 0.1513330191373825, 'KNK'], ['AuRED_040', 'A0', '1265649630110507010', 2, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1265626365400961024', 3, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1266324977382555650', 4, 0.12719853222370148, 'KNK'], ['AuRED_040', 'A0', '1266483508299608072', 5, 0.1193590760231018, 'KNK'], ['AuRED_067', 'A0', '1292482758560948224', 1, 0.18775562942028046, 'KNK'], ['AuRED_067', 'A0', '129214972

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK'], ['AuRED_040', 'A0', '1266672477272408064', 1, 0.1513330191373825, 'KNK'], ['AuRED_040', 'A0', '1265649630110507010', 2, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1265626365400961024', 3, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1266324977382555650', 4, 0.12719853222370148, 'KNK'], ['AuRED_040', 'A0', '1266483508299608072', 5, 0.1193590760231018, 'KNK'], ['AuRED_067', 'A0', '1292482758560948224', 1, 0.18775562942028046, 'KNK'], ['AuRED_067', 'A0', '129214972

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK'], ['AuRED_040', 'A0', '1266672477272408064', 1, 0.1513330191373825, 'KNK'], ['AuRED_040', 'A0', '1265649630110507010', 2, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1265626365400961024', 3, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1266324977382555650', 4, 0.12719853222370148, 'KNK'], ['AuRED_040', 'A0', '1266483508299608072', 5, 0.1193590760231018, 'KNK'], ['AuRED_067', 'A0', '1292482758560948224', 1, 0.18775562942028046, 'KNK'], ['AuRED_067', 'A0', '129214972

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK'], ['AuRED_040', 'A0', '1266672477272408064', 1, 0.1513330191373825, 'KNK'], ['AuRED_040', 'A0', '1265649630110507010', 2, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1265626365400961024', 3, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1266324977382555650', 4, 0.12719853222370148, 'KNK'], ['AuRED_040', 'A0', '1266483508299608072', 5, 0.1193590760231018, 'KNK'], ['AuRED_067', 'A0', '1292482758560948224', 1, 0.18775562942028046, 'KNK'], ['AuRED_067', 'A0', '129214972

[['AuRED_139', 'A0', '1551966453804531712', 1, 0.5743372440338135, 'KNK'], ['AuRED_130', 'A0', '1595690205381791745', 1, 0.17856009304523468, 'KNK'], ['AuRED_007', 'A0', '1230154022855827456', 1, 0.5980465412139893, 'KNK'], ['AuRED_007', 'A0', '1230088352776564736', 2, 0.4988044202327728, 'KNK'], ['AuRED_007', 'A0', '1229795597311451137', 3, 0.31546705961227417, 'KNK'], ['AuRED_147', 'A0', '1570088736259338240', 1, 0.27363285422325134, 'KNK'], ['AuRED_068', 'A0', '1300332268595146752', 1, 0.1329103708267212, 'KNK'], ['AuRED_040', 'A0', '1266672477272408064', 1, 0.1513330191373825, 'KNK'], ['AuRED_040', 'A0', '1265649630110507010', 2, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1265626365400961024', 3, 0.13456904888153076, 'KNK'], ['AuRED_040', 'A0', '1266324977382555650', 4, 0.12719853222370148, 'KNK'], ['AuRED_040', 'A0', '1266483508299608072', 5, 0.1193590760231018, 'KNK'], ['AuRED_067', 'A0', '1292482758560948224', 1, 0.18775562942028046, 'KNK'], ['AuRED_067', 'A0', '129214972

In [5]:
Evidence_retrieval_output = pd.DataFrame(EVLIST, columns = ['id', 'Dummy', 'timeline_id', 'rank', 'predictions', 'Team'])
Evidence_retrieval_output.to_csv('./results/Evidence_retrieval_output.csv', index=False)
Evidence_retrieval_output

,id,Dummy,timeline_id,rank,predictions,Team
0,AuRED_139,A0,1551966453804531712,1,0.574337,KNK
1,AuRED_130,A0,1595690205381791745,1,0.178560,KNK
2,AuRED_007,A0,1230154022855827456,1,0.598047,KNK
3,AuRED_007,A0,1230088352776564736,2,0.498804,KNK
4,AuRED_007,A0,1229795597311451137,3,0.315467,KNK
5,AuRED_147,A0,1570088736259338240,1,0.273633,KNK
6,AuRED_068,A0,1300332268595146752,1,0.132910,KNK
7,AuRED_040,A0,1266672477272408064,1,0.151333,KNK
8,AuRED_040,A0,1265649630110507010,2,0.134569,KNK
9,AuRED_040,A0,1265626365400961024,3,0.134569,KNK


In [6]:
Rumor_Verification_output = pd.DataFrame(FLIST, columns = ['id', 'predicted_label', 'claim', 'predicted_evidence'])
Rumor_Verification_output.to_csv('./results/Rumor_Verification_output.csv', index=False)
Rumor_Verification_output

,id,predicted_label,claim,predicted_evidence
0,AuRED_139,SUPPORTS,The leak of the mouse in the Al-Ahly and El-Go...,"[[https://twitter.com/EFA, 1551966453804531712..."
1,AuRED_151,NOT ENOUGH INFO,Journalist Samer Jawad: Millions of dollars di...,"[[https://twitter.com/iraqmedianet , 156745120..."
2,AuRED_130,REFUTES,#Jeddah_now. Hospitals flooded in Jeddah https...,"[[https://twitter.com/Jeddahhealth, 1595690205..."
3,AuRED_007,SUPPORTS,#Qatar announces the first infection with the ...,"[[https://twitter.com/HMC_Qatar, 1230154022855..."
4,AuRED_147,REFUTES,Russian Sputnik: Turkish intelligence informs ...,"[[https://twitter.com/SyrianCoalition, 1570088..."
5,AuRED_005,NOT ENOUGH INFO,Where was the salt born? Urgent: Chief of Staf...,"[[https://twitter.com/KuwaitArmyGHQ, 117545256..."
6,AuRED_068,REFUTES,#Saudi Arabia did not agree to allow the first...,"[[https://twitter.com/IsraelArabic, 1300332268..."
7,AuRED_040,SUPPORTS,"My brothers and sisters, if one of you has sym...","[[https://twitter.com/UNICCairo, 1266672477272..."
8,AuRED_067,SUPPORTS,Saad Hariri announces that Egypt’s aid has exp...,"[[https://twitter.com/LBpresidency, 1292482758..."
9,AuRED_084,NOT ENOUGH INFO,#Urgent: Declaring a state of emergency in the...,"[[https://twitter.com/ADMediaOffice, 126099460..."
